## Download NYC Taxi data

### Imports

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
!pip install geo-py

In [ ]:
import IPython.core.debugger as db
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import time 

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import glob
from PIL import Image

from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import mean_squared_log_error, mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn import metrics
from sklearn import mixture

import geo.sphere 

%matplotlib inline

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (12,8)
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

### Data Paths

In [ ]:
data_path = Path.cwd()/'nyc'

### Kaggle Data Download

In [ ]:
def kaggle_data(data_type, data_name):
  # Run this cell and select the kaggle.json file downloaded
  # from the Kaggle account settings page.
  from google.colab import files
  files.upload()

  # Let's make sure the kaggle.json file is present.
  !ls -lha kaggle.json

  # Next, install the Kaggle API client after forcing an upgrade
  !pip uninstall -y kaggle
  !pip install --upgrade pip
  !pip install kaggle==1.5.6
  !kaggle -v

  # Reason for doing a force-upgrade. The underlying problem: Colab installs both py2 and py3 
  # packages, and (for historical reasons) the py2 packages are installed second. kaggle is a 
  # wrapper installed by the kaggle python package; since we do py2 second, the py2 wrapper 
  # is in /usr/local/bin, and happens to be an older version.

  # The Kaggle API client expects this file to be in ~/.kaggle,
  # so move it there.
  !mkdir -p ~/.kaggle
  !cp kaggle.json ~/.kaggle/

  # This permissions change avoids a warning on Kaggle tool startup.
  !chmod 600 ~/.kaggle/kaggle.json

  # List available datasets.
  !kaggle {data_type} list

  # First, you have to login to Kaggle, go to that competition's page, navigate to 
  # the Rules tab and accept the terms and conditions. Unless you do that, you will get
  # a 403-Forbidden error when you run the command below

  # Copy the carvana data set locally.
  !kaggle {data_type} download {data_name}

In [ ]:
kaggle_data_type = 'competitions'
kaggle_data_name = 'nyc-taxi-trip-duration'
kaggle_data(kaggle_data_type, kaggle_data_name)

In [ ]:
def nyc_extract(zip_file, out_path):
  !zipinfo {zip_file}

  !unzip {zip_file} -d {out_path} >> /dev/null
  !unzip {out_path}/'train.zip' -d {out_path} >> /dev/null
  !unzip {out_path}/'test.zip' -d {out_path} >> /dev/null
  !ls -l {out_path}

nyc_extract(f'{kaggle_data_name}.zip', data_path)

### Load Data

In [ ]:
train_file = data_path/'train.csv'
df = pd.read_csv(train_file)
df.shape
df.head()

## Geo Location NYC Taxi data with Light GBM - [article](https://medium.com/analytics-vidhya/feature-engineering-with-geospatial-data-predicting-nyc-cab-trip-duration-a121ec16021b), [notebook](https://github.com/claudian37/DS_Portfolio/blob/master/NYC_cab_dataset/02_Modeling_NYC_Cab_final.ipynb)

### Explore Target 

In [ ]:
target='trip_duration'
df[target].mean()
sns.kdeplot(df[target])

In [ ]:
# Take Log Transformation of Target LabelT. Trip durations are highly skewed (left histogram), 
# with values concentrated near zero and a few outliers of long trips
fig, ax = plt.subplots(1,2)
fig.set_size_inches(12,6)
foo = ax[0].hist(df[target], bins=30, label='original', color='red')
foo = ax[1].hist(np.log1p(df[target]), bins=30, label='log')
foo = fig.suptitle(target)
foo = fig.legend()

In [ ]:
df['trip_duration_log'] = np.log1p(df['trip_duration'])
df.head()

### Nulls

In [ ]:
# There are no NULL data values
null_summary = pd.concat((df.isnull().sum(), df.isnull().sum()/df.shape[0]), axis=1)
null_summary.columns = ['actual', 'pct']
null_summary['dtype'] = df.dtypes
null_summary

### Model - Baseline

In [ ]:
# RMSLE for predicting mean of trip duration
np.sqrt(mean_squared_log_error(df['trip_duration'], np.repeat(df['trip_duration'].mean(), df.shape[0])))

# RMSLE for predicting mean of log of trip duration
np.sqrt(mean_squared_log_error(np.exp(df['trip_duration_log']), np.exp(np.repeat(df['trip_duration_log'].mean(), df.shape[0]))))

In [ ]:
target = 'trip_duration_log'

numerical_cols = df.dtypes[df.dtypes != 'object'].index
# Remove the target columns
numerical_cols = numerical_cols.drop(['trip_duration_log', 'trip_duration'])

# Naive model - No feature engineering; just modeling with cross validation. Use a small sample of 10k observations to see how random forest does
rf_cv_mean = np.sqrt(-cross_val_score(estimator=RandomForestRegressor(),
                X=df[numerical_cols].sample(10000, random_state=1), 
                y=df[target].sample(10000, random_state=1),
                scoring='neg_mean_squared_error').mean())
print(f'Random Forest CV Mean RMSLE: {rf_cv_mean}')

### Feature Engineering

In [ ]:
# Time-based Features - transform pickup_datetime column into features by month, day, day_of_week, hour_of_day, minute_of_hour.
# Use holidays package to create binary feature to flag dates as holidays in NY state.

import holidays
class FeaturizeTime(object):
    def fit(self, X, y):
        return self
        
    def transform(self, X):
        X = X.copy()
        # Engineer temporal features
        X['pickup_datetime'] = pd.to_datetime(X['pickup_datetime'])
        X['month'] = X['pickup_datetime'].dt.month
        X['day'] = X['pickup_datetime'].dt.day
        X['day_of_week'] = X['pickup_datetime'].dt.dayofweek
        X['hour_of_day'] = X['pickup_datetime'].dt.hour
        X['minute_of_hour'] = X['pickup_datetime'].dt.minute
        
        # Get holidays in NY state in 2016
        us_holidays = holidays.US(state='NY', years=2016)
        X['is_holiday'] = X['pickup_datetime'].dt.date.apply(lambda x: 1 if x in us_holidays.keys() else 0)
        
        # Drop time column in final df
        X = X.drop(['pickup_datetime'], axis=1)
        print('Time-based features transformed')
        return X

In [ ]:
# Calculate Distance Between Coordinates using geo-py package:
# Great Circle Distance between pickup and dropoff locations (great_circle_distance): this is the shortest distance between two points on a sphere.
# Manhattan Distance between pickup and dropoff locations (manhattan_distance)
# Bearing between pickup and dropoff locations (bearing): this is the direction from the pickup location to dropoff location.

class CalculateDistances(object):
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        X = X.copy()
        
        ## 1) Find min distance between pickup and dropoff coordinates 
        X['great_circle_distance'] = X.apply(lambda x: self._calculate_great_circle_distance(x['pickup_latitude'], x['pickup_longitude'], 
                                                                       x['dropoff_latitude'], x['dropoff_longitude']),
                                                                       axis=1)
        
        ## 2) Calculate manhattan distance between pickup and dropoff coordinates
        X['manhattan_distance'] = X.apply(lambda x: self._calculate_manhattan_distance(x['pickup_latitude'], x['pickup_longitude'], 
                                                                       x['dropoff_latitude'], x['dropoff_longitude']),
                                                                       axis=1)
        
        ## 3) Calculate bearing between pickup and dropoff latitude
        X['bearing'] = X.apply(lambda x: self._calculate_bearing(x['pickup_latitude'], x['pickup_longitude'], 
                                                           x['dropoff_latitude'], x['dropoff_longitude']),
                                                           axis=1)
        print('Distance features calculated')
        return X
    
    def _calculate_great_circle_distance(self, pickup_lat, pickup_long, dropoff_lat, dropoff_long):
        pickup = [pickup_lat, pickup_long]
        dropoff = [dropoff_lat, dropoff_long]
        distance = geo.sphere.distance(pickup, dropoff)
        return distance
    
    def _calculate_manhattan_distance(self, pickup_lat, pickup_long, dropoff_lat, dropoff_long):
        pickup = [pickup_lat, pickup_long]
        dropoff_a = [pickup_lat, dropoff_long]
        dropoff_b = [dropoff_lat, pickup_long]
        distance_a = geo.sphere.distance(pickup, dropoff_a)
        distance_b = geo.sphere.distance(pickup, dropoff_b)
        return distance_a + distance_b
        
    def _calculate_bearing(self, pickup_lat, pickup_long, dropoff_lat, dropoff_long):
        pickup = [pickup_lat, pickup_long]
        dropoff = [dropoff_lat, dropoff_long]
        bearing = geo.sphere.bearing(pickup, dropoff)
        return bearing

In [ ]:
# Cluster Density - estimate traffic density at the different pickup and dropoff locations.
# Use mini-batch K-Means to group our pickup and dropoff locations into 20 distinct clusters 
# Get the number of pickups/dropoffs at each cluster per time interval (per hour)
# Compute the percentage of rides in each pickup/ dropoff cluster per day.

class GetClusterDensity():
    def __init__(self, n_clusters):
        self.n_clusters = n_clusters
        self.pickup_time_clusters = pd.DataFrame()
        self.dropoff_time_clusters = pd.DataFrame()
    
    def fit(self, X, y):
        # I. Fit pickup cluster
        df_pickup = X[['pickup_latitude', 'pickup_longitude']].copy()
        
        ## Ia. Initialize K-means 
        self.clf_pickup = MiniBatchKMeans(n_clusters=self.n_clusters, batch_size=10000, max_iter=300, random_state=1)
        self.clf_pickup.fit_predict(df_pickup)
        df_pickup['pickup_cluster'] = self.clf_pickup.labels_
        df_pickup['pickup_cluster'] = df_pickup['pickup_cluster'].astype(str)
        X = pd.merge(X, df_pickup[['pickup_cluster']], left_index=True, right_index=True, how='left')
        self.clf_pickup.fit(df_pickup.drop(['pickup_cluster'], axis=1))
        
        ## Ib. Calculate number of rides grouped by cluster and time as proxy for "density" of traffic by location and time
        pickup_time_clusters = pd.DataFrame(X.groupby(['month', 'day', 'hour_of_day', 'pickup_cluster'])['pickup_latitude'].count()).reset_index()
        pickup_time_clusters = pickup_time_clusters.rename(columns={'pickup_latitude': 'num_rides_by_pickup_group'})
        pickup_time_clusters_agg = pd.DataFrame(pickup_time_clusters.reset_index().groupby(['month', 'day'])['num_rides_by_pickup_group'].sum().round(4))
        pickup_time_clusters_agg = pickup_time_clusters_agg.rename(columns={'num_rides_by_pickup_group': 'agg_rides_per_day'})
        pickup_time_clusters = pd.merge(pickup_time_clusters.set_index(['month', 'day', 'hour_of_day']), pickup_time_clusters_agg, left_index=True, right_index=True)
        pickup_time_clusters['perc_rides_per_day_by_pickup_group'] = pickup_time_clusters['num_rides_by_pickup_group']/pickup_time_clusters['agg_rides_per_day']
        pickup_time_clusters = pickup_time_clusters.reset_index()
        pickup_time_clusters['pickup_group'] = pickup_time_clusters['pickup_cluster'].map(str) + str(',') + \
                            pickup_time_clusters['month'].map(str) + str(',') + pickup_time_clusters['day'].map(str) + \
                            str(',') + pickup_time_clusters['hour_of_day'].map(str)
        self.pickup_time_clusters = pickup_time_clusters
        
        
        # II. Fit dropoff cluster
        df_dropoff = X[['dropoff_latitude', 'dropoff_longitude']].copy()       
        
        ## IIa. Initialize K-means 
        self.clf_dropoff = MiniBatchKMeans(n_clusters=20, batch_size=10000, max_iter=300, random_state=1)
        self.clf_dropoff.fit_predict(df_dropoff)
        df_dropoff['dropoff_cluster'] = self.clf_dropoff.labels_
        df_dropoff['dropoff_cluster'] = df_dropoff['dropoff_cluster'].astype(str)
        X = pd.merge(X, df_dropoff[['dropoff_cluster']], left_index=True, right_index=True, how='left')
        self.clf_dropoff.fit(df_dropoff.drop(['dropoff_cluster'], axis=1))
        
        ## IIb. Calculate number of rides grouped by cluster and time as proxy for "density" of traffic by location and time
        dropoff_time_clusters = pd.DataFrame(X.groupby(['month', 'day', 'hour_of_day', 'dropoff_cluster'])['dropoff_latitude'].count()).reset_index()
        dropoff_time_clusters = dropoff_time_clusters.rename(columns={'dropoff_latitude': 'num_rides_by_dropoff_group'})
        dropoff_time_clusters_agg = pd.DataFrame(dropoff_time_clusters.reset_index().groupby(['month', 'day'])['num_rides_by_dropoff_group'].sum().round(4))
        dropoff_time_clusters_agg = dropoff_time_clusters_agg.rename(columns={'num_rides_by_dropoff_group': 'agg_rides_per_day'})
        dropoff_time_clusters = pd.merge(dropoff_time_clusters.set_index(['month', 'day', 'hour_of_day']), dropoff_time_clusters_agg, left_index=True, right_index=True)
        dropoff_time_clusters['perc_rides_per_day_by_dropoff_group'] = dropoff_time_clusters['num_rides_by_dropoff_group']/dropoff_time_clusters['agg_rides_per_day']
        dropoff_time_clusters = dropoff_time_clusters.reset_index()
        dropoff_time_clusters['dropoff_group'] = dropoff_time_clusters['dropoff_cluster'].map(str) + str(',') + \
                            dropoff_time_clusters['month'].map(str) + str(',') + dropoff_time_clusters['day'].map(str) + \
                            str(',') + dropoff_time_clusters['hour_of_day'].map(str)
        self.dropoff_time_clusters = dropoff_time_clusters
        
        return self
    
    def transform(self, X):
        # III. Predict pickup cluster
        df_pickup = X[['pickup_latitude', 'pickup_longitude']].copy()
        
        ## IIIa. Add cluster label
        df_pickup['pickup_cluster'] = self.clf_pickup.predict(df_pickup)
        df_pickup['pickup_cluster'] = df_pickup['pickup_cluster'].astype(str)
        
        ## IIIb. Merge cluster label back to original dataframe
        X = pd.merge(X, df_pickup[['pickup_cluster']], left_index=True, right_index=True, how='left')
        
        ## IIIc. Merge to create "num_rides_by_pickup_group" and "perc_rides_by_pickup_group" features
        X['pickup_group'] = X['pickup_cluster'].map(str) + str(',') + X['month'].map(str) + str(',') + \
                            X['day'].map(str) + str(',') + X['hour_of_day'].map(str)
        X = pd.merge(X, self.pickup_time_clusters[['pickup_group','num_rides_by_pickup_group', 'perc_rides_per_day_by_pickup_group']], on='pickup_group', how='left')
        X = X.drop(['pickup_group', 'pickup_cluster'], axis=1)
        print('Pickup Clusters Found')
        
        
        # IV. Predict dropoff cluster
        df_dropoff = X[['dropoff_latitude', 'dropoff_longitude']].copy() 
        
        ## IVa. Add cluster label
        df_dropoff['dropoff_cluster'] = self.clf_dropoff.predict(df_dropoff)
        df_dropoff['dropoff_cluster'] = df_dropoff['dropoff_cluster'].astype(str)
        
        ## IVb. Merge cluster label back to original dataframe
        X = pd.merge(X, df_dropoff[['dropoff_cluster']], left_index=True, right_index=True, how='left')
        
        ## IVc. Merge to create "num_rides_by_pickup_group" and "perc_rides_by_pickup_group" features
        X['dropoff_group'] = X['dropoff_cluster'].map(str) + str(',') + X['month'].map(str) + str(',') + \
                             X['day'].map(str) + str(',') + X['hour_of_day'].map(str)
        X = pd.merge(X, self.dropoff_time_clusters[['dropoff_group','num_rides_by_dropoff_group', 'perc_rides_per_day_by_dropoff_group']], on='dropoff_group', how='left')
        X = X.drop(['dropoff_group', 'dropoff_cluster'], axis=1)
        print('Dropoff Clusters Found')
        
        return X

In [ ]:
# K-Means clusters didn’t add much on top of lat/long coordinates
# Instead, use a probabilistic Gaussian Mixture Model (GMM) model - GMM learns the 
# probabilities of a data point belonging to each pickup/dropoff cluster. groups 
# of pickup and dropoff locations. So a point is not uniquely assigned to a single
# cluster, but can reside in multiple cluster with some probability.

class GetClusterProbability():
    def __init__(self, n_components):
        self.n_components = n_components
    
    # First scale the pickup and dropoff locations using StandardScaler(), then fit 
    # a GMM model of 20 components on the train set 
    def fit(self, X, y):
        # Pickup locations
        self.pickup_cols = ['pickup_latitude', 'pickup_longitude']
        self.scaler_pickup = StandardScaler()
        pickup_scaled = self.scaler_pickup.fit_transform(X[self.pickup_cols])
        self.gmm_pickup = mixture.GaussianMixture(n_components=self.n_components).fit(pickup_scaled)
        
        # Dropoff locations
        self.dropoff_cols = ['dropoff_latitude', 'dropoff_longitude']
        self.scaler_dropoff = StandardScaler()
        dropoff_scaled = self.scaler_dropoff.fit_transform(X[self.dropoff_cols])
        self.gmm_dropoff = mixture.GaussianMixture(n_components=self.n_components).fit(dropoff_scaled)
        
        return self
    
    def transform(self, X):
        # Pickup locations
        pickup_scaled = self.scaler_pickup.transform(X[self.pickup_cols])
        preds_pickup = pd.DataFrame(self.gmm_pickup.predict_proba(pickup_scaled))
        preds_pickup = preds_pickup.add_prefix('gmm_pickup_')
        X = pd.merge(X, preds_pickup, left_index=True, right_index=True)
        print('GMM for pickup done')
        
        # Dropoff locations
        dropoff_scaled = self.scaler_dropoff.transform(X[self.dropoff_cols])
        preds_dropoff = pd.DataFrame(self.gmm_dropoff.predict_proba(dropoff_scaled))
        preds_dropoff = preds_dropoff.add_prefix('gmm_dropoff_')
        X = pd.merge(X, preds_dropoff, left_index=True, right_index=True)
        print('GMM for dropoff done')
        
        return X

In [ ]:
# One-hot Encode categorical features - the vendor_id and store_and_fwd_flag columns, 
# both of which only have two unique values

class DummifyCategoricals():
    def fit(self, X, y):
        self.cols_to_encode=['vendor_id', 'store_and_fwd_flag']
        self.encoder = OneHotEncoder(categories='auto', handle_unknown='ignore')
        
        transformed_array = self.encoder.fit_transform(X[self.cols_to_encode]).toarray()
        self.transformed_colnames = [f'{prefix}_{value}' 
                                     for prefix, values in zip(self.cols_to_encode, self.encoder.categories_) 
                                     for value in values]
            
        return self
        
    def transform(self, X):
        transformed_array = self.encoder.transform(X[self.cols_to_encode]).toarray()
        transformed_df = pd.DataFrame(transformed_array, columns=self.transformed_colnames)
        
        X = pd.concat((X.drop(self.cols_to_encode, axis=1).reset_index(drop=True), 
                       transformed_df.reset_index(drop=True)), axis=1)
        print('Categorical variables dummified')
        return X

In [ ]:
# Split into train, valid and test
train, df_test = train_test_split(df)
df_train, df_valid = train_test_split(train)
df_train.shape, df_valid.shape, df_test.shape

# Remove original target from df_train and df_test
df_train = df_train.drop(['trip_duration'], axis=1)
df_valid = df_valid.drop(['trip_duration'], axis=1)
df_test = df_test.drop(['trip_duration'], axis=1)

In [ ]:
# Use sklearn make_pipeline() to construct a half pipeline to transform features.

initial_feats = ['vendor_id',
                 'pickup_datetime',
                 'passenger_count',
                 'pickup_longitude',
                 'pickup_latitude',
                 'dropoff_longitude',
                 'dropoff_latitude',
                 'store_and_fwd_flag']

half_pipeline = make_pipeline(
    FeaturizeTime(),
    CalculateDistances(),
    GetClusterDensity(n_clusters=20),
    GetClusterProbability(n_components=20),
    DummifyCategoricals()
    )

df_train_feat = half_pipeline.fit_transform(df_train[initial_feats])
df_valid_feat = half_pipeline.transform(df_valid[initial_feats])
df_test_feat = half_pipeline.transform(df_test[initial_feats])

df_train_feat.head()

### Model - training using Light GBM

In [ ]:
# Evaluation metric - use rmse rather than rmsle, as we are taking log() of the target values
def calc_rmsle(y_true_log, y_preds_log):
    return np.sqrt(mean_squared_log_error(np.exp(y_true_log), np.exp(y_preds_log)))

# Light GBM model
lgb = LGBMRegressor()
lgb.fit(df_train_feat, df_train[target], eval_set=[(df_valid_feat, df_valid[target])], 
        eval_metric='rmse', early_stopping_rounds=10, verbose=1)

### Model - Prediction

In [ ]:
y_preds_lgbm = lgb.predict(df_test_feat)
calc_rmsle(df_test[target], y_preds_lgbm)

### Feature Selection

In [ ]:
# Use top 50 features ranked by feature importance

best_feats = df_train_feat.columns[np.argsort(lgb.feature_importances_)[::-1]]
best_feats

In [ ]:
features_intersection = [col for col in df_train_feat[df_train_feat.columns.intersection(df_test_feat.columns)].columns]
features_intersection
len(df_train_feat.columns), len(df_test_feat.columns), len(features_intersection)

In [ ]:
best_feats_intersection = [feat for feat in best_feats if feat in features_intersection]
print(len(best_feats_intersection))
best_feats_intersection

In [ ]:
train_scores = []
test_scores = []

for i in range(1, len(best_feats_intersection)):
    lgb.fit(df_train_feat.loc[:, best_feats_intersection[:i]], df_train[target])
    train_preds = lgb.predict(df_train_feat.loc[:, best_feats_intersection[:i]])
    train_scores.append(calc_rmsle(df_train[target],train_preds))
    test_preds = lgb.predict(df_test_feat.loc[:, best_feats_intersection[:i]])
    test_scores.append(calc_rmsle(df_test[target], test_preds))
    print(f'RMSLE recorded for round {i}')

In [ ]:
plt.plot(range(1, len(best_feats_intersection)), train_scores, label='train RMSLE')
plt.plot(range(1, len(best_feats_intersection)), test_scores, label='test RMSLE')

plt.xticks(np.arange(1,len(best_feats_intersection),2))
plt.xlabel('num features used in LGBM Regressor Model')
plt.ylabel('RMSLE Score')
plt.legend()
plt.title('Forward Feature Selection')
#plt.ylim(0.39,0.45)
#plt.savefig('./plots/step_forward_feat_sel.png', bbox_inches='tight')

In [ ]:
min(test_scores)
best_n_feats = test_scores.index(min(test_scores)) + 1
print ('Num features', best_n_feats)

In [ ]:
X_train = df_train_feat.loc[:, best_feats_intersection[:best_n_feats]]
X_valid = df_valid_feat.loc[:, best_feats_intersection[:best_n_feats]]
X_test = df_test_feat.loc[:, best_feats_intersection[:best_n_feats]]

y_train = df_train[target]
y_valid = df_valid[target]
y_test = df_test[target]
X_train.shape, X_valid.shape, X_test.shape

### Hyperparam Tuning

In [ ]:
# Use GridSearch to find the optimal hyperparameters for LightGBM, like max_depth and num_leaves

params = {
    'num_leaves': [256,512,1024], 
    'max_depth': [8,10,12], 
    'n_estimators': [500],
    'subsample': [0.8],
    'feature_fraction': [0.9],
    'lambda_l1': [0.2],
    'learning_rate': [0.1]
}

def calc_rmsle(y_true_log, y_preds_log):
    return np.sqrt(mean_squared_log_error(np.exp(y_true_log), np.exp(y_preds_log)))

rmsle_scorer = metrics.make_scorer(calc_rmsle, greater_is_better=False)

lgb = LGBMRegressor(eval_metric='rmse')
reg = GridSearchCV(lgb, params, scoring=rmsle_scorer, verbose=True)
reg.fit(X_train, y_train)

In [ ]:
reg.best_score_

In [ ]:
reg.best_params_

In [ ]:
gb_cv_results = pd.DataFrame.from_dict(reg.cv_results_).sort_values(by='mean_test_score', ascending=False)
gb_cv_results

## Geo Clusters with Geopandas and Shapely - [article 2](https://towardsdatascience.com/finding-and-visualizing-clusters-of-geospatial-data-698943c18fed), [notebook 2](https://github.com/claudian37/DS_Portfolio/blob/master/NYC_cab_dataset/03_visualizing_kmeans_clusters_geospatial_data.ipynb)

In [ ]:
!pip install geopandas

In [ ]:
import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point

%matplotlib inline

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (12,8)

In [ ]:
# Download the shape file for NY boroughs
ny_shape_zip='nydata.zip'
ny_shape_dir=Path.cwd()/'nyshape'

# Put double quotes around the URL so that the query params get passed correctly
!wget -O {ny_shape_zip} "https://data.cityofnewyork.us/api/geospatial/tqmj-j8zm?method=export&format=Shapefile"
!zipinfo {ny_shape_zip}
!unzip {ny_shape_zip} -d {ny_shape_dir} >> /dev/null

# Downloaded filenames are randomly generated, so rename to a known name
!mv nyshape/geo*.shp nyshape/ny.shp
!mv nyshape/geo*.shx nyshape/ny.shx
!mv nyshape/geo*.prj nyshape/ny.prj
!mv nyshape/geo*.dbf nyshape/ny.dbf

!ls nyshape

### Pre-process data

In [ ]:
df_pickup = df[['pickup_latitude', 'pickup_longitude']].copy()
df_pickup = df_pickup.rename(columns={'pickup_latitude': 'latitude', 'pickup_longitude': 'longitude'})

# Drop lat/longs of extreme outliers below 0.01 percentile or above 99.99 percentile
df_pickup_filtered = df_pickup[(df_pickup.quantile(0.0001) < df_pickup) & (df_pickup < df_pickup.quantile(0.9999))]
df_pickup_filtered = df_pickup_filtered.dropna(how='any')

print(f'{df_pickup.shape[0] - df_pickup_filtered.shape[0]} extreme outliers removed')
print(f'Shape of filtered df_pickup: {df_pickup_filtered.shape}')

### Find clusters and plot

In [ ]:
import IPython.core.debugger as db
def get_cluster(df, n_clusters):
    km = MiniBatchKMeans(n_clusters=n_clusters)
    km.fit_predict(df)

    # Calculate sum of squared distances
    ssd = km.inertia_
    
    # Label cluster centers
    centers = km.cluster_centers_
    
    # Get cluster center
    clusters = km.labels_

    return (ssd, centers, clusters)

In [ ]:
def plot_cluster(base_map, gdf, centers, clusters):
  fig, ax = plt.subplots(figsize=(10,10))
  ax.set_aspect('equal')
  base_map.plot(ax=ax, alpha=0.4, edgecolor='darkgrey', color='lightgrey', label=base_map['boro_name'], zorder=1)
  gdf.plot(ax=ax, column=clusters, alpha=0.5, cmap='viridis', linewidth=0.8, zorder=2)
  centers_gseries = GeoSeries([Point(centers[i, 1], centers[i, 0]) for i in range(centers.shape[0])])
  centers_gseries.plot(ax=ax, alpha=1, marker='X', color='red', markersize=100, zorder=3)

  n_clusters = len(clusters)
  plt.title(f'K-Means: Pickup Locations grouped into {n_clusters} clusters')
  plt.xlabel('longitude')
  plt.ylabel('latitude')
  plt.show()

In [ ]:
def best_clusters(df, max_clusters):
  ssds = []

  for i in range(2, max_clusters):
    ssd, centers, clusters = get_cluster(df=df, n_clusters=i)
    # Save sum of squared distances
    ssds.append(ssd)

  plt.plot(range(2,max_clusters), ssds)
  plt.xlabel('k')
  plt.ylabel('Sum of Squared Distances')
  plt.title('Elbow Method to Find Optimal Value of k')

In [ ]:
# Create a Geopandas dataframe and plot it
ny_crs = {'init': 'epsg:4326'}
ny_geom = [Point(xy) for xy in zip(df_pickup_filtered['longitude'], df_pickup_filtered['latitude'])]
ny_gdf = gpd.GeoDataFrame(df_pickup_filtered, crs=ny_crs, geometry=ny_geom)
ny_gdf.plot()

In [ ]:
# Load the Shape map of the NYC as the base map
nyc_full = gpd.read_file(ny_shape_dir/'ny.shp')
nyc_full.head()

In [ ]:
# Plot lat/long and clusters on map for some cluster values
_, centers, clusters = get_cluster(df=df_pickup_filtered, n_clusters=3)
plot_cluster(nyc_full, ny_gdf, centers, clusters)

In [ ]:
_, centers, clusters = get_cluster(df=df_pickup_filtered, n_clusters=6)
plot_cluster(nyc_full, ny_gdf, centers, clusters)

In [ ]:
_, centers, clusters = get_cluster(df=df_pickup_filtered, n_clusters=12)
plot_cluster(nyc_full, ny_gdf, centers, clusters)

In [ ]:
best_clusters(df=df_pickup_filtered, max_clusters=15)

## Geo Heatmaps with Folium

In [ ]:
# Install newer version of Folium so that HeatMapWithTime() works
# Colab has 0.8.3 installed by default, whereas we need 0.11.0 or greater
!pip install folium --force-reinstall

In [ ]:
# !!!!!! Delete this cell if the above cell works
!pip uninstall folium
!pip install folium --force-reinstall

In [ ]:
import folium
print ('Folium version:', folium.__version__)
from folium import plugins
from folium.plugins import HeatMap
from IPython.display import IFrame

df_heat = df[['pickup_datetime', 'pickup_longitude', 'pickup_latitude']].copy()
df_heat['day_of_week'] = pd.to_datetime(df_heat['pickup_datetime']).dt.dayofweek
df_heat['hour_of_day'] = pd.to_datetime(df_heat['pickup_datetime']).dt.hour
df_heat.head()

In [ ]:
# Instantiate map object 
map_5 = folium.Map(location=[40.75, -73.96], tiles='openstreetmap', zoom_start=12)

hm_data=[]
ti=[]
for i in range(df_heat.day_of_week.min(), df_heat.day_of_week.max()+1):
    for j in range(df_heat.hour_of_day.min(), df_heat.hour_of_day.max()+1):    
        
        # Filter to include only data for each day of week and hour of day
        df_geo = df_heat.loc[(df_heat.day_of_week==i) & (df_heat.hour_of_day==j)][['pickup_latitude', 'pickup_longitude']].copy()
        hm_data.append(df_geo.to_numpy().tolist())
        ti.append(f'{i}-{j}')

# Plot heatmap - data size is too big, so plot a subset of the heatmaps
# Then click on the animation in the bottom left corner to see the heatmaps over time slots
hm = plugins.HeatMapWithTime(hm_data[:50], index=ti[:50])
hm.add_to(map_5)
map_5

## Interactive Map Visualisations with Folium

In [ ]:
# Uses lat then lon. The bigger the zoom number, the closer in you get
map_hooray = folium.Map(location=[51.5074, 0.1278], zoom_start = 11) 
map_hooray # Calls the map to display

In [ ]:
# Different visual styles
t_list = ["Stamen Terrain", "Stamen Toner", "Mapbox Bright"]
map_hooray = folium.Map(location=[51.5074, 0.1278], tiles = "Stamen Terrain",  zoom_start = 12)
map_hooray

In [ ]:
# Different visual styles
map_hooray = folium.Map(location=[51.5074, 0.1278], tiles = "Stamen Toner", zoom_start = 12)
map_hooray

In [ ]:
# Markers - once the base map is defined, everything else gets added on top of it with map.add_to()
# Note the 'popup attribute. This text appears on clicking the map.
# 'width=int' and 'height=int' can be in pixels or percentages
map_hooray = folium.Map(location=[51.5074, 0.1278], width=600, height=500, zoom_start = 12)
folium.Marker([51.5079, 0.0877], popup='London Bridge').add_to(map_hooray)
map_hooray

In [ ]:
# More complex markers
# The London Bridge marker is a pin icon marker, with an extra line of green.
# The second marker is a coloured overlay marker. CircleMarker radius is set in pixels so if you change the zoom you need to change the pixels. It can also take a fill_color that's semi-transparent.
# The third marker is interactive - so the user can click anywhere multiple times to add their own markers.

# Set the map up
map_hooray = folium.Map(location=[51.5074, 0.1278], width=600, height=500, zoom_start = 9)
# Simple marker
folium.Marker([51.5079, 0.0877], popup='London Bridge', icon=folium.Icon(color='green')).add_to(map_hooray)
# Circle marker
folium.CircleMarker([51.4183, 0.2206], radius=30, popup='East London', color='red').add_to(map_hooray)
# Interactive marker
map_hooray.add_child(folium.ClickForMarker(popup="Folium is awesome"))

map_hooray

In [ ]:
# Adds tool to the top right
from folium import plugins
from folium.plugins import MeasureControl
from folium.plugins import FloatImage

# Interactivity
map_hooray = folium.Map(location=[51.5074, 0.1278], width=600, height=500, zoom_start = 11) 
# Click on the map if you don't see the measure control icon in the top right
map_hooray.add_child(MeasureControl())
url = ('https://media.licdn.com/mpr/mpr/shrinknp_100_100/AAEAAQAAAAAAAAlgAAAAJGE3OTA4YTdlLTkzZjUtNDFjYy1iZThlLWQ5OTNkYzlhNzM4OQ.jpg')
FloatImage(url, bottom=5, left=85).add_to(map_hooray)

map_hooray

In [ ]:
# Add icons from fontawesome.io, by referencing the "prefix='fa'""

map_hooray = folium.Map(location=[51.5074, 0.1278], width=600, height=500, zoom_start = 9)

folium.Marker([51.5079, 0.0877], popup='London Bridge', icon=folium.Icon(color='green')).add_to(map_hooray)
folium.Marker([51.5183, 0.5206], popup='East London', icon=folium.Icon(color='red',icon='university', prefix='fa')).add_to(map_hooray)
folium.Marker([51.5183, 0.3206], popup='East London', icon=folium.Icon(color='blue',icon='bar-chart', prefix='fa')).add_to(map_hooray)
             # icon=folium.Icon(color='red',icon='bicycle', prefix='fa')
map_hooray.add_child(folium.ClickForMarker(popup="Folium is awesome"))

map_hooray